In [1]:
import pandas as pd
import numpy as np
import json

import time
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

### 1.0 Scraping **The Edge Malaysia**

For [The Edge Malaysia](https://theedgemalaysia.com/), each of their articles seem to have a unique ID, e.g., "https://theedgemalaysia.com/node/677590". Hence, since we won't be able to do this by month, page no., etc., we'll use a **brute force** approach that tests every combination of numbers, such that we'll only scrape from a valid url.

In [2]:
headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
    }

In [3]:
# function to get text and metadata from url
def process_url(x):
    while True:
        webpage = f'https://theedgemalaysia.com/node/{x}'
        try:
            r = requests.get(webpage, headers=headers)
            break
        except Exception as e:
            time.sleep(5.0)
    
    soup = BeautifulSoup(r.text, "lxml")

    try:
        headline = soup.find('title').text
        h = soup.find('div', class_="news-detail_newsTextDataWrap__PkAu5") 
        content_list = [p.text for p in h.find_all("p")]
        content_str = ' '.join(content_list)
        if 'English version' in content_str:
            language = 'Mandarin'
        else:
            language = 'English'

    except Exception as e:
        return None

    data = {'url': f'https://theedgemalaysia.com/node/{x}', 
            'headline': headline,
            'language': language,
            'content': content_str}

    return data

In [4]:
np.random.seed(10082023)
batch1 = [str(x) for x in np.random.randint(0, 100000, size=20000)]
batch2 = [str(x) for x in np.random.randint(100001, 200000, size=20000)]
batch3 = [str(x) for x in np.random.randint(200001, 300000, size=20000)]
batch4 = [str(x) for x in np.random.randint(300001, 400000, size=20000)]
batch5 = [str(x) for x in np.random.randint(400001, 500000, size=20000)]
batch6 = [str(x) for x in np.random.randint(500001, 600000, size=20000)]
batch7 = [str(x) for x in np.random.randint(600001, 700000, size=20000)]

# let's just try batch1 first.
batches = [batch2, batch3, batch4]

In [5]:
# consolidate links and get texts
max_workers = 30

for i, urls in enumerate(batches):

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_url, x) for x in urls]

        for future in tqdm(futures, total=len(urls)):
            result = future.result()
            if result:
                with open(f'theedgemalaysia--complete-batch-{i+2}.jsonl', 'a') as f:
                    json.dump(result, f)
                    f.write('\n')

  0%|          | 0/20000 [00:00<?, ?it/s]

### 2.0 Final checks

In [6]:
test1 = pd.read_json('theedgemalaysia--complete-batch-1.jsonl', lines=True)
print(f'No. of articles in batch 1: {len(test1)}')
print(test1.head(3))

No. of articles: 19902
                                      url  \
0  https://theedgemalaysia.com/node/33730   
1  https://theedgemalaysia.com/node/71781   
2  https://theedgemalaysia.com/node/20831   

                                            headline language  \
0     Corporate: Tough times ahead for steel players  English   
1  IIA urges mandatory adoption of internal audit...  English   
2                                  Privatisation 2.0  English   

                                             content  
0  Higher raw material costs and uncertainties ov...  
1  KUALA LUMPUR: The Institute of Internal Audito...  
2  ONE of the policy objectives desired to be ach...  
